# 1. Настройка гиперпараметров Tf-IDF Vectorizer



*   **stopwords** - удаление стоп-слов
*   **analyzer** ('word', 'char') - "единица для векторизации"
*  **lowercaser** (True, False) - переводим ли текст в нижний регистр
*  **ngram_range** - диапазон n-грамм
* **max_df** - верхний порог частотности
* **min_df** - нижний порого частотности
* **use_idf** - используем ли IDF
* **norm** (None, 'l1', 'l2') - нормализация Tf-IDF вектора



# 2. Настройка гиперпараметров RandomForest



*   **n_estimators** - количество деревьев
*   **max_depth** - глубина дерева
* **min_samples_split** - минимальное число образцов
* **max_samples_split** - максимальное число образцов
* **criterion** ('gini', 'entropy') - функция, по которой алгоритм оценивает качество разбиения узлов в деревьях



# 3. Подготавливаем датасет AG News

In [2]:
!pip install datasets
from datasets import load_dataset
dataset_news = load_dataset("ag_news")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is i

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [3]:
import random

count0, count1, count2, count3 = 0, 0, 0, 0
dataset_short_news = []
for i in range(len(dataset_news['train'])):
  if dataset_news['train'][i]['label'] == 0 and count0 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_news['train'][i]['label'] == 1 and count1 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 1})
    count1 += 1
  elif dataset_news['train'][i]['label'] == 2 and count2 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 2})
    count2 += 1
  elif dataset_news['train'][i]['label'] == 3 and count3 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 3})
    count3 += 1

random.shuffle(dataset_short_news)
dataset_news = {'train': dataset_short_news[:6400], 'test': dataset_short_news[6400:]}

In [4]:
train_texts_news = [s['news'] for s in dataset_news['train']]
test_texts_news = [s['news'] for s in dataset_news['test']]

train_labels_news = [s['label'] for s in dataset_news['train']]
test_labels_news = [s['label'] for s in dataset_news['test']]

# Выводим примеры
print(train_texts_news[0], train_labels_news[0])
print(test_texts_news[0], test_labels_news[0])

People flee storm-ravaged Philippine towns as rescuers search for &lt;b&gt;...&lt;/b&gt; Panicked residents scrambled to flee three storm-ravaged coastal Philippine towns on Sunday while rescuers made last-ditch attempts to find survivors of the powerful storm that left more than 1,100 dead and missing. 0
Ford dumps \$200m Oracle system Ford has ditched Oracle #39;s online purchasing system eVEREST, because of flaws in the software. The car manufacturer has reportedly spent more than \$200m trying to get the system to work since it bought it in 2000.  2


In [5]:
# из предобработки сделаю только токенезацию и удаление спец.символов
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
import re

def preprocess(text):
  text = text.lower()
  tokens = word_tokenize(text)
  tokens = [re.sub(r'[^\w\s]', '', word) for word in tokens]

  return ' '.join(tokens)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [6]:
train_texts_news = [preprocess(i) for i in train_texts_news]

# 4. Настройка гиперпараметров randomForest и Tf-IDF через GridSearchCV (полный перебор)

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import pandas as pd

In [8]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", RandomForestClassifier(random_state=42))
])

In [23]:
param_grid = {
    "tfidf__stop_words": ["english"],
    "tfidf__ngram_range": [(1,1), (1,2)],
    "tfidf__analyzer": ["word"],
    "tfidf__lowercase": [True],
    "tfidf__max_df": [0.7,0.8, 0.95],
    "tfidf__min_df": [1, 3],
    'tfidf__use_idf': [True, False],
    "tfidf__norm": [None, 'l1', 'l2'],

    "clf__n_estimators": [100, 200, 250],
    "clf__max_depth": [None, 30],
    "clf__min_samples_split": [2, 5, 7],
}

In [24]:
res_news = []

for s_w in param_grid["tfidf__stop_words"]:
  for ng in param_grid['tfidf__ngram_range']:
    for analyz in param_grid['tfidf__analyzer']:
      for low in param_grid['tfidf__lowercase']:
        for max_df in param_grid['tfidf__max_df']:
          for min_df in param_grid['tfidf__min_df']:
            for use_idf in param_grid['tfidf__use_idf']:
              for n in param_grid['tfidf__norm']:
                for n_est in param_grid['clf__n_estimators']:
                  for max_d in param_grid['clf__max_depth']:
                    for min_s in param_grid['clf__min_samples_split']:
                      pipeline.set_params(
                          tfidf__stop_words = s_w,
                          tfidf__ngram_range = ng,
                          tfidf__analyzer = analyz,
                          tfidf__lowercase = low,
                          tfidf__max_df = max_df,
                          tfidf__min_df = min_df,
                          tfidf__use_idf = use_idf,
                          tfidf__norm = n,
                          clf__n_estimators = n_est,
                          clf__max_depth = max_d,
                          clf__min_samples_split = min_s
                      )

                      pipeline.fit(train_texts_news, train_labels_news)
                      y_pred_news = pipeline.predict(test_texts_news)
                      f1 = f1_score(test_labels_news, y_pred_news, average='weighted')

                      curr_news = [s_w, ng, analyz, low, max_df, min_df, use_idf, n, n_est, max_d, min_s, f1]
                      res_news.append(curr_news)
                      if len(res_news) % 10 == 0:
                        print(len(res_news))

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290


In [25]:
df_results = pd.DataFrame(res_news, columns=["stopwords", "ngram_range", "analyzer", "lowercase", "max_df", "min_df", "use_idf", "norm", "n_estimator", "max_depth", "min_samples_split", 'f1-score'])
df_results.to_excel("results_news_tfidf_rf.xlsx", index=False)

Делаем то же самое для логистической регрессии

In [13]:
pipeline_lr = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", LogisticRegression())
])

In [26]:
param_grid_lr = {
    "tfidf__stop_words": ["english"],
    "tfidf__ngram_range": [(1,1), (1,2)],
    "tfidf__analyzer": ["word"],
    "tfidf__lowercase": [True],
    "tfidf__max_df": [0.7, 0.8, 0.95],
    "tfidf__min_df": [1, 3],
    'tfidf__use_idf': [True, False],
    "tfidf__norm": [None, 'l1', 'l2'],

    "clf__penalty": ['l1', 'l2'],
    "clf__C": [0.01, 0.001, 0.0001, 1],  # сила регуляризации
    "clf__solver": ['liblinear', 'saga'],
    "clf__class_weight": ['balanced', None]
}

In [27]:
res_news_lr = []

for sw in param_grid_lr['tfidf__stop_words']:
  for ng in param_grid_lr['tfidf__ngram_range']:
    for analyz in param_grid_lr['tfidf__analyzer']:
      for low in param_grid_lr['tfidf__lowercase']:
        for max_df in param_grid_lr['tfidf__max_df']:
          for min_df in param_grid_lr['tfidf__min_df']:
            for use_idf in param_grid_lr['tfidf__use_idf']:
              for n in param_grid_lr['tfidf__norm']:
                for pen in param_grid_lr['clf__penalty']:
                  for c in param_grid_lr['clf__C']:
                    for sol in param_grid_lr['clf__solver']:
                      for cw in param_grid_lr['clf__class_weight']:
                          pipeline_lr.set_params(
                            tfidf__stop_words = sw,
                            tfidf__ngram_range = ng,
                            tfidf__analyzer = analyz,
                            tfidf__lowercase = low,
                            tfidf__max_df = max_df,
                            tfidf__min_df = min_df,
                            tfidf__use_idf = use_idf,
                            tfidf__norm = n,
                            clf__penalty = pen,
                            clf__C = c,
                            clf__solver = sol,
                            clf__class_weight = cw
                          )

                          pipeline_lr.fit(train_texts_news, train_labels_news)
                          y_pred_news = pipeline_lr.predict(test_texts_news)
                          f1 = f1_score(test_labels_news, y_pred_news, average='weighted')

                          curr_news = [sw, ng, analyz, low, max_df, min_df, use_idf, n, pen, c, sol, cw, f1]
                          res_news_lr.append(curr_news)
                          if len(res_news_lr) % 10 == 0:
                            print(len(res_news_lr))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


10


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


20
30


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


...

2240
2250


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


2260
2270
2280


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


2290
2300


In [28]:
df_results = pd.DataFrame(res_news_lr, columns=["stopwords", "ngram_range", "analyzer", "lowercase", "max_df", "min_df", "use_idf", "norm", "penalty", "C", "solver", "class_weight", 'f1-score'])
df_results.to_excel("results_news_tfidf_lr1.xlsx", index=False)